This notebook compares analytical and numerical evaluations of alchemical derivatives of a CH3OH molecule up to the third order in Hartree-Fock theory.  
The results are summerized in the last section.

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf

pyscf.__version__

'2.2.1'

In [2]:
from FcMole import FcM_like

In [3]:
target_mol = """
C	-1.27549516	0.03357710	0.00000000
H	-2.10170657	-0.87840808	1.59107621
H	-2.10170657	-0.87840808	-1.59107621
H	-1.76544024	1.99528228	0.00000000
O	1.41253482	-0.22423026	0.00000000
H	2.19050133	1.29874374	0.00000000
"""
# Coordinates in Angstrom
# C	-0.67496302	0.01776824	0.00000000
# H	-1.11217530	-0.46483357	0.84196133
# H	-1.11217530	-0.46483357	-0.84196133
# H	-0.93423081	1.05585799	0.00000000
# O	0.74748129	-0.11865755	0.00000000
# H	1.15916347	0.68726564	0.00000000
dft_functional = "pbe0"  # "lda,vwn"
basis_set = "def2-TZVP"

### Analytical evaluation

In [4]:
%load_ext autoreload
from AP_class import APDFT_perturbator as AP

In [5]:
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
mf_nn=scf.RHF(mol_NN)
# mf_nn.xc = dft_functional
mf_nn.scf()
# mf_nn.conv_tol = 1e-12
# mf_nn.grids.level = 4
# mf_nn.nuc_grad_method().kernel()
ap_nn=AP(mf_nn,sites=[0,1,2,3,4,5],flag_response_property=True)

converged SCF energy = -115.087746889332


In [6]:
analytical_1st_alchemical_derivative = ap_nn.build_gradient()
analytical_2nd_alchemical_derivative = ap_nn.build_hessian()
analytical_3rd_alchemical_derivative = ap_nn.build_cubic()

Inefficient alchemical force is not implemented in the Hartree-Fock theory

Inefficient alchemical force in HFT which calculates the response matrix for nuclear coordinates

In [7]:
# inefficient_analytical_alchemical_force = np.zeros((6, 6, 3))
# for i in range(6):  inefficient_analytical_alchemical_force[i] = ap_nn.build_inefficient_alchemical_force(i)

Efficient alchemical force in HFT which calculates the response matrix for nuclear coordinates

In [8]:
efficient_analytical_alchemical_force = np.zeros((6, 6, 3))
for i in range(6):
  efficient_analytical_alchemical_force[i] = ap_nn.build_alchemical_force(i)

Electric dipole derivatives $\partial ^2 E/ \partial Z_I \partial \mathbf{F}$

In [9]:
elec_electric_dipole_gradient_z = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='Z')
elec_electric_dipole_gradient_f = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='F')

Electric polarizability derivatives $\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F}$

In [10]:
elec_electric_pol_gradient = ap_nn.build_electric_polarizability_gradient()

### Numerical evaluation

In [11]:
%autoreload 2
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
fc_param = 0.001
fcs1 = [fc_param, 0.0, 0.0, 0.0, 0.0, 0.0]
fcs2 = [-fc_param, 0.0, 0.0, 0.0, 0.0, 0.0]
fcs3 = [0.0, fc_param, 0.0, 0.0, 0.0, 0.0]
fcs4 = [0.0, -fc_param, 0.0, 0.0, 0.0, 0.0]
fcs5 = [0.0, 0.0, fc_param, 0.0, 0.0, 0.0]
fcs6 = [0.0, 0.0, -fc_param, 0.0, 0.0, 0.0]
fcs7 = [0.0, 0.0, 0.0, fc_param, 0.0, 0.0]
fcs8 = [0.0, 0.0, 0.0, -fc_param, 0.0, 0.0]
fcs9 = [0.0, 0.0, 0.0, 0.0, fc_param, 0.0]
fcs10 = [0.0, 0.0, 0.0, 0.0, -fc_param, 0.0]
fcs11 = [0.0, 0.0, 0.0, 0.0, 0.0, fc_param]
fcs12 = [0.0, 0.0, 0.0, 0.0, 0.0, -fc_param]

##### Efficient numerical alchemical force

In [12]:
fmol1=FcM_like(mol_NN,fcs=fcs1)
fmol2=FcM_like(mol_NN,fcs=fcs2)
fmol3=FcM_like(mol_NN,fcs=fcs3)
fmol4=FcM_like(mol_NN,fcs=fcs4)
fmol5=FcM_like(mol_NN,fcs=fcs5)
fmol6=FcM_like(mol_NN,fcs=fcs6)
fmol7=FcM_like(mol_NN,fcs=fcs7)
fmol8=FcM_like(mol_NN,fcs=fcs8)
fmol9=FcM_like(mol_NN,fcs=fcs9)
fmol10=FcM_like(mol_NN,fcs=fcs10)
fmol11=FcM_like(mol_NN,fcs=fcs11)
fmol12=FcM_like(mol_NN,fcs=fcs12)
mf=scf.RHF(mol_NN)
mf1=scf.RHF(fmol1)
mf2=scf.RHF(fmol2)
mf3=scf.RHF(fmol3)
mf4=scf.RHF(fmol4)
mf5=scf.RHF(fmol5)
mf6=scf.RHF(fmol6)
mf7=scf.RHF(fmol7)
mf8=scf.RHF(fmol8)
mf9=scf.RHF(fmol9)
mf10=scf.RHF(fmol10)
mf11=scf.RHF(fmol11)
mf12=scf.RHF(fmol12)
# mf.xc = dft_functional
# mf1.xc = dft_functional
# mf2.xc = dft_functional
# mf3.xc = dft_functional
# mf4.xc = dft_functional
# mf5.xc = dft_functional
# mf6.xc = dft_functional
# mf7.xc = dft_functional
# mf8.xc = dft_functional
# mf9.xc = dft_functional
# mf10.xc = dft_functional
# mf11.xc = dft_functional
# mf12.xc = dft_functional
# mf.conv_tol = 1e-12
# mf1.conv_tol = 1e-12
# mf2.conv_tol = 1e-12
# mf3.conv_tol = 1e-12
# mf4.conv_tol = 1e-12
# mf.grids.level = 6
# mf1.grids.level = 6
# mf2.grids.level = 6
# mf3.grids.level = 6
# mf4.grids.level = 6
# Without dm0=mf.init_guess_by_1e(), some SCFs do not converge.
e=mf.scf(dm0=mf.init_guess_by_1e())
e1=mf1.scf(dm0=mf1.init_guess_by_1e())
e2=mf2.scf(dm0=mf2.init_guess_by_1e())
e3=mf3.scf(dm0=mf3.init_guess_by_1e())
e4=mf4.scf(dm0=mf4.init_guess_by_1e())
e5=mf5.scf(dm0=mf5.init_guess_by_1e())
e6=mf6.scf(dm0=mf6.init_guess_by_1e())
e7=mf7.scf(dm0=mf7.init_guess_by_1e())
e8=mf8.scf(dm0=mf8.init_guess_by_1e())
e9=mf9.scf(dm0=mf9.init_guess_by_1e())
e10=mf10.scf(dm0=mf10.init_guess_by_1e())
e11=mf11.scf(dm0=mf11.init_guess_by_1e())
e12=mf12.scf(dm0=mf12.init_guess_by_1e())

converged SCF energy = -115.087746889331
converged SCF energy = -115.10244394927


/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -115.073052811911
converged SCF energy = -115.088877459041
converged SCF energy = -115.086617404318
converged SCF energy = -115.088877459041
converged SCF energy = -115.086617404318
converged SCF energy = -115.088866074659
converged SCF energy = -115.086628787927
converged SCF energy = -115.110098300794
converged SCF energy = -115.065399081023
converged SCF energy = -115.088765014768
converged SCF energy = -115.086729721173


In [13]:
nuc_grad = mf.nuc_grad_method().kernel()
nuc_grad1 = mf1.nuc_grad_method().kernel()
nuc_grad2 = mf2.nuc_grad_method().kernel()
nuc_grad3 = mf3.nuc_grad_method().kernel()
nuc_grad4 = mf4.nuc_grad_method().kernel()
nuc_grad5 = mf5.nuc_grad_method().kernel()
nuc_grad6 = mf6.nuc_grad_method().kernel()
nuc_grad7 = mf7.nuc_grad_method().kernel()
nuc_grad8 = mf8.nuc_grad_method().kernel()
nuc_grad9 = mf9.nuc_grad_method().kernel()
nuc_grad10 = mf10.nuc_grad_method().kernel()
nuc_grad11 = mf11.nuc_grad_method().kernel()
nuc_grad12 = mf12.nuc_grad_method().kernel()

--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0267046097    -0.0036400606     0.0000000000
1 H    -0.0009154969     0.0028092302    -0.0169322993
2 H    -0.0009154969     0.0028092302     0.0169322993
3 H     0.0069442222    -0.0081741117     0.0000000000
4 O     0.0385846229     0.0475486277    -0.0000000000
5 H    -0.0169932415    -0.0413529157     0.0000000000
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0266749234    -0.0036490025    -0.0000000000
1 H    -0.0009539105     0.0027732816    -0.0168698156
2 H    -0.0009539105     0.0027732816     0.0168698156
3 H     0.0069252327    -0.0080949845    -0.0000000000
4 O     0.0386499654     0.0475609708     0.0000000000
5 H    -0.0169924536    -0.0413635469    -0.0000000000
----------------------------------------------
--------------- RHF gradients ---------------
         x        

In [14]:
efficient_numerical_alchemical_force = np.zeros((6, 6, 3))
efficient_numerical_alchemical_force[0] = (nuc_grad1 - nuc_grad2) / (2 * fc_param)
efficient_numerical_alchemical_force[1] = (nuc_grad3 - nuc_grad4) / (2 * fc_param)
efficient_numerical_alchemical_force[2] = (nuc_grad5 - nuc_grad6) / (2 * fc_param)
efficient_numerical_alchemical_force[3] = (nuc_grad7 - nuc_grad8) / (2 * fc_param)
efficient_numerical_alchemical_force[4] = (nuc_grad9 - nuc_grad10) / (2 * fc_param)
efficient_numerical_alchemical_force[5] = (nuc_grad11 - nuc_grad12) / (2 * fc_param)

#### Numerical alchemical derivatives

In [15]:
numerical_1st_alchemical_derivative = np.zeros((6))
numerical_1st_alchemical_derivative[0] = (e1 - e2) / (2 * fc_param)
numerical_1st_alchemical_derivative[1] = (e3 - e4) / (2 * fc_param)
numerical_1st_alchemical_derivative[2] = (e5 - e6) / (2 * fc_param)
numerical_1st_alchemical_derivative[3] = (e7 - e8) / (2 * fc_param)
numerical_1st_alchemical_derivative[4] = (e9 - e10) / (2 * fc_param)
numerical_1st_alchemical_derivative[5] = (e11 - e12) / (2 * fc_param)

#### Numerical alchemical hardness

In [16]:
ap_nn=AP(mf,sites=[0,1,2,3,4,5])
ap_nn1=AP(mf1,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn2=AP(mf2,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn3=AP(mf3,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn4=AP(mf4,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn5=AP(mf5,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn6=AP(mf6,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn7=AP(mf7,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn8=AP(mf8,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn9=AP(mf9,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn10=AP(mf10,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn11=AP(mf11,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn12=AP(mf12,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
an = ap_nn.build_gradient()
an1 = ap_nn1.build_gradient()
an2 = ap_nn2.build_gradient()
an3 = ap_nn3.build_gradient()
an4 = ap_nn4.build_gradient()
an5 = ap_nn5.build_gradient()
an6 = ap_nn6.build_gradient()
an7 = ap_nn7.build_gradient()
an8 = ap_nn8.build_gradient()
an9 = ap_nn9.build_gradient()
an10 = ap_nn10.build_gradient()
an11 = ap_nn11.build_gradient()
an12 = ap_nn12.build_gradient()
ann = ap_nn.build_hessian()
ann1 = ap_nn1.build_hessian()
ann2 = ap_nn2.build_hessian()
ann3 = ap_nn3.build_hessian()
ann4 = ap_nn4.build_hessian()
ann5 = ap_nn5.build_hessian()
ann6 = ap_nn6.build_hessian()
ann7 = ap_nn7.build_hessian()
ann8 = ap_nn8.build_hessian()
ann9 = ap_nn9.build_hessian()
ann10 = ap_nn10.build_hessian()
ann11 = ap_nn11.build_hessian()
ann12 = ap_nn12.build_hessian()
dipole1 = ap_nn1.ref_elec_electric_dipole_moment * -1
dipole2 = ap_nn2.ref_elec_electric_dipole_moment * -1
dipole3 = ap_nn3.ref_elec_electric_dipole_moment * -1
dipole4 = ap_nn4.ref_elec_electric_dipole_moment * -1
dipole5 = ap_nn5.ref_elec_electric_dipole_moment * -1
dipole6 = ap_nn6.ref_elec_electric_dipole_moment * -1
dipole7 = ap_nn7.ref_elec_electric_dipole_moment * -1
dipole8 = ap_nn8.ref_elec_electric_dipole_moment * -1
dipole9 = ap_nn9.ref_elec_electric_dipole_moment * -1
dipole10 = ap_nn10.ref_elec_electric_dipole_moment * -1
dipole11 = ap_nn11.ref_elec_electric_dipole_moment * -1
dipole12 = ap_nn12.ref_elec_electric_dipole_moment * -1
pol1 = ap_nn1.ref_elec_electric_polarizability * -1
pol2 = ap_nn2.ref_elec_electric_polarizability * -1
pol3 = ap_nn3.ref_elec_electric_polarizability * -1
pol4 = ap_nn4.ref_elec_electric_polarizability * -1
pol5 = ap_nn5.ref_elec_electric_polarizability * -1
pol6 = ap_nn6.ref_elec_electric_polarizability * -1
pol7 = ap_nn7.ref_elec_electric_polarizability * -1
pol8 = ap_nn8.ref_elec_electric_polarizability * -1
pol9 = ap_nn9.ref_elec_electric_polarizability * -1
pol10 = ap_nn10.ref_elec_electric_polarizability * -1
pol11 = ap_nn11.ref_elec_electric_polarizability * -1
pol12 = ap_nn12.ref_elec_electric_polarizability * -1

In [17]:
numerical_2nd_alchemical_derivative = np.zeros((6, 6))
numerical_2nd_alchemical_derivative[0] = (an1 - an2) / (2 * fc_param)
numerical_2nd_alchemical_derivative[1] = (an3 - an4) / (2 * fc_param)
numerical_2nd_alchemical_derivative[2] = (an5 - an6) / (2 * fc_param)
numerical_2nd_alchemical_derivative[3] = (an7 - an8) / (2 * fc_param)
numerical_2nd_alchemical_derivative[4] = (an9 - an10) / (2 * fc_param)
numerical_2nd_alchemical_derivative[5] = (an11 - an12) / (2 * fc_param)

#### Numerical alchemical hardness derivatives

In [18]:
numerical_3rd_alchemical_derivative = np.zeros((6, 6, 6))
numerical_3rd_alchemical_derivative[0, :, :] = (ann1 - ann2) / (2 * fc_param)
numerical_3rd_alchemical_derivative[1, :, :] = (ann3 - ann4) / (2 * fc_param)
numerical_3rd_alchemical_derivative[2, :, :] = (ann5 - ann6) / (2 * fc_param)
numerical_3rd_alchemical_derivative[3, :, :] = (ann7 - ann8) / (2 * fc_param)
numerical_3rd_alchemical_derivative[4, :, :] = (ann9 - ann10) / (2 * fc_param)
numerical_3rd_alchemical_derivative[5, :, :] = (ann11 - ann12) / (2 * fc_param)

#### Inefficient numerical alchemical force

In [19]:
# mol_NN1=gto.M(atom="C 0 0 0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN2=gto.M(atom="C 0 0 -0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN3=gto.M(atom="C 0 0 0; O 0 0 2.101",unit="Bohr",basis=basis_set)
# mol_NN4=gto.M(atom="C 0 0 0; O 0 0 2.099",unit="Bohr",basis=basis_set)
# mf_nn1=scf.RKS(mol_NN1)
# mf_nn2=scf.RKS(mol_NN2)
# mf_nn3=scf.RKS(mol_NN3)
# mf_nn4=scf.RKS(mol_NN4)
# mf_nn1.xc = dft_functional
# mf_nn2.xc = dft_functional
# mf_nn3.xc = dft_functional
# mf_nn4.xc = dft_functional
# mf_nn1.scf()
# mf_nn2.scf()
# mf_nn3.scf()
# mf_nn4.scf()
# ap_nn1=AP(mf_nn1,sites=[0,1])
# ap_nn2=AP(mf_nn2,sites=[0,1])
# ap_nn3=AP(mf_nn3,sites=[0,1])
# ap_nn4=AP(mf_nn4,sites=[0,1])
# ad1 = ap_nn1.build_gradient()
# ad2 = ap_nn2.build_gradient()
# ad3 = ap_nn3.build_gradient()
# ad4 = ap_nn4.build_gradient()

In [20]:
# inefficient_numerical_alchemical_force = np.zeros((2, 2))  # only z axis components
# inefficient_numerical_alchemical_force[:, 0] = (ad1 - ad2) / (2 * fc_param)
# inefficient_numerical_alchemical_force[:, 1] = (ad3 - ad4) / (2 * fc_param)

#### Numerical alchemical electric dipole moment

In [21]:
numer_elec_electric_dipole_gradient = np.zeros((6, 3))
numer_elec_electric_dipole_gradient[0] = (dipole1 - dipole2) / (2 * fc_param)
numer_elec_electric_dipole_gradient[1] = (dipole3 - dipole4) / (2 * fc_param)
numer_elec_electric_dipole_gradient[2] = (dipole5 - dipole6) / (2 * fc_param)
numer_elec_electric_dipole_gradient[3] = (dipole7 - dipole8) / (2 * fc_param)
numer_elec_electric_dipole_gradient[4] = (dipole9 - dipole10) / (2 * fc_param)
numer_elec_electric_dipole_gradient[5] = (dipole11 - dipole12) / (2 * fc_param)

#### Numerical alchemical electric polarizability

In [22]:
numer_elec_electric_pol_gradient = np.zeros((6, 3, 3))
numer_elec_electric_pol_gradient[0] = (pol1 - pol2) / (2 * fc_param)
numer_elec_electric_pol_gradient[1] = (pol3 - pol4) / (2 * fc_param)
numer_elec_electric_pol_gradient[2] = (pol5 - pol6) / (2 * fc_param)
numer_elec_electric_pol_gradient[3] = (pol7 - pol8) / (2 * fc_param)
numer_elec_electric_pol_gradient[4] = (pol9 - pol10) / (2 * fc_param)
numer_elec_electric_pol_gradient[5] = (pol11 - pol12) / (2 * fc_param)

## Comparison of analytical and numerical evaluations

$ \partial E/\partial Z_i $

In [23]:
print(analytical_1st_alchemical_derivative)

[-14.69556852  -1.13002724  -1.13002724  -1.11864327 -22.3496097
  -1.01764698]


$ (\partial E/\partial Z_i)_\mathrm{analytical} $ - $ (\partial E/\partial Z_i)_\mathrm{numerical} $

In [24]:
print(analytical_1st_alchemical_derivative - numerical_1st_alchemical_derivative)

[ 1.56531915e-07  1.21314558e-07  1.21300353e-07  9.54678123e-08
  1.87597983e-07 -1.86552136e-07]


$\partial^2E/\partial Z_i\partial Z_j$

In [25]:
print(analytical_2nd_alchemical_derivative)

[[-2.98251879  0.40535361  0.40535361  0.40313263  0.30192006  0.27684405]
 [ 0.40535361 -1.08469606  0.32594434  0.33120622  0.28192602  0.26925656]
 [ 0.40535361  0.32594434 -1.08469606  0.33120622  0.28192602  0.26925656]
 [ 0.40313263  0.33120622  0.33120622 -1.08392396  0.27796653  0.27601045]
 [ 0.30192006  0.28192602  0.28192602  0.27796653 -3.60315557  0.44664312]
 [ 0.27684405  0.26925656  0.26925656  0.27601045  0.44664312 -0.95727943]]


$(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{numerical}$

In [26]:
print(analytical_2nd_alchemical_derivative - numerical_2nd_alchemical_derivative)

[[ 7.39182608e-07 -7.12461388e-08 -7.12514726e-08  1.30484634e-07
  -5.54984972e-07 -2.79695145e-07]
 [ 8.56204422e-07  5.68431971e-07 -6.27590335e-07 -1.00980450e-08
  -1.44541892e-07 -2.75700870e-07]
 [ 8.56204417e-07 -6.27596553e-07  5.68425762e-07 -1.00922724e-08
  -1.44547224e-07 -2.75707978e-07]
 [ 7.58257134e-07 -1.21585753e-07 -1.21595079e-07  2.49853234e-08
  -1.39891417e-07  7.65886338e-08]
 [ 4.73200564e-07 -2.83530559e-08 -2.83606079e-08 -6.05642059e-08
  -6.46092539e-08 -8.10458457e-08]
 [ 4.68570748e-07 -3.69950368e-07 -3.69947261e-07 -7.75115851e-07
   1.59181548e-07 -4.98535759e-08]]


$\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [27]:
print(analytical_3rd_alchemical_derivative)

[[[-0.26923386  0.08290763  0.08290763  0.08173221  0.04068338
    0.03735554]
  [ 0.08290763  0.11297092  0.075523    0.07688749  0.04816134
    0.04270145]
  [ 0.08290763  0.075523    0.11297092  0.07688749  0.04816134
    0.04270145]
  [ 0.08173221  0.07688749  0.07688749  0.10926678  0.04615448
    0.0386318 ]
  [ 0.04068338  0.04816134  0.04816134  0.04615448  0.02184993
    0.01430208]
  [ 0.03735554  0.04270145  0.04270145  0.0386318   0.01430208
    0.02408054]]

 [[ 0.08290763  0.11297092  0.075523    0.07688749  0.04816134
    0.04270145]
  [ 0.11297092 -0.85251327  0.07055889  0.07111204  0.05262468
    0.04236894]
  [ 0.075523    0.07055889  0.07055889  0.03669421  0.02811253
    0.02466812]
  [ 0.07688749  0.07111204  0.03669421  0.07004581  0.02764435
    0.02278396]
  [ 0.04816134  0.05262468  0.02811253  0.02764435  0.02931532
    0.02123612]
  [ 0.04270145  0.04236894  0.02466812  0.02278396  0.02123612
    0.03413624]]

 [[ 0.08290763  0.075523    0.11297092  0.076887

$(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{analytical}$ - $(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{numerical}$

In [28]:
print(analytical_3rd_alchemical_derivative - numerical_3rd_alchemical_derivative)

[[[ 3.07488298e-07  1.58695962e-07  1.58693408e-07 -2.25700553e-08
   -2.24296106e-08 -2.09427898e-07]
  [ 1.58695962e-07  1.68132948e-07  3.22043167e-08 -3.93984564e-08
    8.12472483e-08 -2.35882064e-08]
  [ 1.58693408e-07  3.22043167e-08  1.68137505e-07 -3.93985113e-08
    8.12455831e-08 -2.35896494e-08]
  [-2.25700553e-08 -3.93984564e-08 -3.93985113e-08 -9.63573115e-08
   -4.03279268e-08 -3.90618896e-08]
  [-2.24296106e-08  8.12472483e-08  8.12455831e-08 -4.03279268e-08
   -3.15030975e-07 -1.11445580e-07]
  [-2.09427898e-07 -2.35882064e-08 -2.35896494e-08 -3.90618896e-08
   -1.11445580e-07 -1.78225211e-07]]

 [[ 1.93434896e-07  1.03075723e-06 -9.00709346e-07 -6.27486393e-08
   -1.91415046e-08 -1.45174503e-07]
  [ 1.03075723e-06  1.03909408e-06  3.87180414e-08  2.02525059e-07
    1.70171642e-07  1.58468049e-07]
  [-9.00709346e-07  3.87180413e-08 -9.65259552e-07 -2.87951544e-07
   -2.08807865e-07 -2.72505828e-07]
  [-6.27486393e-08  2.02525060e-07 -2.87951544e-07 -1.18234879e-07
   -

$\partial^2E/\partial Z_i\partial R_j$

In [29]:
print(efficient_analytical_alchemical_force)

[[[ 2.96917535e-02 -8.93827017e-03 -3.60981208e-16]
  [-3.84227481e-02 -3.59576124e-02  6.24986218e-02]
  [-3.84227481e-02 -3.59576124e-02 -6.24986218e-02]
  [-1.89953474e-02  7.91455308e-02 -1.28286134e-16]
  [ 6.53609181e-02  1.23331496e-02 -2.44443424e-16]
  [ 7.88171970e-04 -1.06251856e-02  2.72906423e-17]]

 [[-3.92178651e-02  3.82388140e-03  1.08009098e-02]
  [-3.63509304e-02 -2.93281689e-02  4.14559011e-02]
  [ 3.07357587e-03  6.05029424e-03 -1.85598798e-02]
  [ 7.15033669e-03  1.36558467e-02 -1.60707974e-02]
  [ 6.81798525e-02  1.21173141e-02 -1.20929105e-02]
  [-2.83496949e-03 -6.31916755e-03 -5.53322332e-03]]

 [[-3.92178651e-02  3.82388140e-03 -1.08009098e-02]
  [ 3.07357587e-03  6.05029424e-03  1.85598798e-02]
  [-3.63509304e-02 -2.93281689e-02 -4.14559011e-02]
  [ 7.15033669e-03  1.36558467e-02  1.60707974e-02]
  [ 6.81798525e-02  1.21173141e-02  1.20929105e-02]
  [-2.83496949e-03 -6.31916755e-03  5.53322332e-03]]

 [[-3.40243707e-02  7.61357308e-03  5.52717806e-16]
  [ 2.

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [30]:
print(efficient_analytical_alchemical_force - efficient_numerical_alchemical_force)

[[[ 4.55705909e-08  8.75813501e-08  1.19974345e-12]
  [-1.40337216e-07 -4.39494842e-08  3.71305725e-08]
  [-1.40335882e-07 -4.39494279e-08 -3.71322365e-08]
  [ 1.30286610e-07  2.99067288e-08  1.45882431e-13]
  [-6.83426997e-08  1.71568340e-07  9.55029970e-14]
  [ 1.73154641e-07 -2.01143102e-07 -1.28382957e-13]]

 [[ 8.11400154e-08  6.54037633e-08 -1.34287936e-08]
  [-7.71739443e-07 -2.55169470e-07  7.55310435e-07]
  [ 5.62025113e-07  1.27622878e-07  5.62466490e-07]
  [ 1.01991650e-08 -1.11015452e-08  3.85481074e-08]
  [-8.20490650e-08  3.51795545e-07 -2.05182471e-06]
  [ 2.00424619e-07 -2.78554278e-07  7.08927929e-07]]

 [[ 8.11426824e-08  6.54060683e-08  1.34297825e-08]
  [ 5.62026335e-07  1.27620492e-07 -5.62467158e-07]
  [-7.71741549e-07 -2.55170080e-07 -7.55303774e-07]
  [ 1.02014419e-08 -1.11026549e-08 -3.85482502e-08]
  [-8.20415183e-08  3.51793105e-07  2.05182533e-06]
  [ 2.00426729e-07 -2.78556277e-07 -7.08928063e-07]]

 [[-1.53481738e-07 -5.37721043e-08 -4.69211833e-13]
  [-4.

$\partial^2E/\partial Z_i\partial R_j$

In [31]:
# print(inefficient_analytical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [32]:
# print(inefficient_analytical_alchemical_force[:, :, 2] - inefficient_numerical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical,efficient}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical,inefficient}$

In [33]:
# print(efficient_analytical_alchemical_force - inefficient_analytical_alchemical_force)

$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$

In [34]:
print(elec_electric_dipole_gradient_f)

[[-6.95770662e-02 -4.94637258e-02  2.01976787e-15]
 [-8.14181657e-01 -7.33278122e-01  1.16414205e+00]
 [-8.14181657e-01 -7.33278122e-01 -1.16414205e+00]
 [-5.37633116e-01  1.41616267e+00  2.99776527e-15]
 [ 8.84766638e-01 -2.29857064e-01 -4.42959032e-15]
 [ 1.44046214e+00  9.99961311e-01 -1.32271518e-15]]


In [35]:
print(elec_electric_dipole_gradient_z)

[[-6.95770660e-02 -4.94637303e-02  8.31287034e-15]
 [-8.14181653e-01 -7.33278116e-01  1.16414206e+00]
 [-8.14181653e-01 -7.33278116e-01 -1.16414206e+00]
 [-5.37633058e-01  1.41616266e+00  3.88568853e-16]
 [ 8.84766682e-01 -2.29857066e-01 -3.93730975e-15]
 [ 1.44046211e+00  9.99961297e-01 -1.61345041e-15]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$ - $(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{numerical}$

In [36]:
print(elec_electric_dipole_gradient_f - numer_elec_electric_dipole_gradient)

[[ 6.08718772e-07  1.45905845e-06 -4.04672487e-13]
 [-7.12865943e-07  1.61528205e-06 -1.10451438e-05]
 [-7.12880679e-07  1.61526385e-06  1.10451414e-05]
 [-1.28061220e-06  2.39335248e-06 -2.03628016e-12]
 [ 5.07601144e-07 -4.58539005e-07  9.92368581e-13]
 [-2.25421875e-06  3.17761180e-06 -4.64008547e-12]]


In [37]:
print(elec_electric_dipole_gradient_z - numer_elec_electric_dipole_gradient)

[[ 6.08951086e-07  1.45462299e-06 -3.98379384e-13]
 [-7.08705168e-07  1.62149990e-06 -1.10343150e-05]
 [-7.08719909e-07  1.62148171e-06  1.10343126e-05]
 [-1.22183487e-06  2.38606894e-06 -2.03888936e-12]
 [ 5.52207311e-07 -4.60290931e-07  9.92860861e-13]
 [-2.27996933e-06  3.16338107e-06 -4.64037621e-12]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$

In [38]:
print(elec_electric_pol_gradient)

[[[ 1.55300779e+00  5.20240386e-01 -1.01597402e-13]
  [ 5.20240386e-01  5.34009192e+00 -3.09355066e-13]
  [-1.01597402e-13 -3.09355066e-13  5.39764612e+00]]

 [[ 3.76535404e+00  9.22795296e-01 -7.54006038e-01]
  [ 9.22795296e-01  6.30660148e+00 -9.47413118e-01]
  [-7.54006038e-01 -9.47413118e-01  7.49289527e+00]]

 [[ 3.76535404e+00  9.22795296e-01  7.54006038e-01]
  [ 9.22795296e-01  6.30660148e+00  9.47413118e-01]
  [ 7.54006038e-01  9.47413118e-01  7.49289527e+00]]

 [[ 3.81412903e+00 -2.05196128e-02 -1.03859444e-13]
  [-2.05196128e-02  7.98650160e+00 -4.39919446e-13]
  [-1.03859444e-13 -4.39919446e-13  5.62183256e+00]]

 [[ 6.61238448e+00  5.16808867e-02 -2.30907494e-13]
  [ 5.16808867e-02  4.86118643e+00 -2.15278417e-13]
  [-2.30907494e-13 -2.15278417e-13  5.04611341e+00]]

 [[ 4.92011962e+00 -2.47565066e-01 -1.60486566e-13]
  [-2.47565066e-01  3.50877382e+00 -1.72318917e-13]
  [-1.60486566e-13 -1.72318917e-13  3.66089866e+00]]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$ - $(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{numerical}$

In [39]:
print(elec_electric_pol_gradient - numer_elec_electric_pol_gradient)

[[[ 1.86161602e-05  3.48239463e-06  5.85306977e-11]
  [ 3.48239463e-06 -1.92472790e-05 -5.60778035e-11]
  [ 5.85306977e-11 -5.60778035e-11 -9.96524186e-06]]

 [[ 1.91937402e-05  7.93701861e-07 -4.33822924e-05]
  [ 7.93701861e-07 -3.17760031e-05 -1.96748987e-05]
  [-4.33822924e-05 -1.96748987e-05 -1.17773652e-05]]

 [[ 1.91935484e-05  7.93688505e-07  4.33823583e-05]
  [ 7.93688505e-07 -3.17761064e-05  1.96748686e-05]
  [ 4.33823583e-05  1.96748686e-05 -1.17780150e-05]]

 [[ 1.47776479e-05  6.99735270e-06  9.37864363e-11]
  [ 6.99735270e-06 -4.58633407e-05  1.16059180e-10]
  [ 9.37864363e-11  1.16059180e-10 -1.83187486e-05]]

 [[ 2.62692469e-05  2.61989369e-06 -2.29463119e-10]
  [ 2.61989369e-06  1.12187794e-05 -2.34061870e-10]
  [-2.29463119e-10 -2.34061870e-10 -2.00341976e-05]]

 [[ 7.42552682e-05 -5.09958457e-06 -2.19692228e-11]
  [-5.09958457e-06  7.52500951e-06  5.31322958e-12]
  [-2.19692228e-11  5.31322958e-12 -1.87730692e-05]]]
